In [1]:
import random
import json
import numpy as np
import tensorflow as tf
import pandas as pd
import re
import pickle

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
intents = json.loads(open('./intents.json').read())

intents_df = pd.read_json('./intents.json')

In [3]:
# Flatten data
df_chat = pd.json_normalize(intents, record_path =['intents'])
df_chat = df_chat[['tag', 'patterns']]
df_chat = df_chat.explode(['patterns'], ignore_index=True)
df_chat = pd.DataFrame(df_chat)

In [4]:
classes = sorted(df_chat['tag'].unique())

In [5]:
lemmatizer = WordNetLemmatizer()
tokenizer = word_tokenize
vectorizer_bow = CountVectorizer()

In [6]:
def lemm(word):
    words = tokenizer(word)
    words = [lemmatizer.lemmatize(text) for text in words]
    return words

In [7]:
def bagofword(text):
    word = lemm(text)
    bag = vectorizer_bow.transform(word)
    return bag.toarray()
    
    

In [8]:
# def predict(t):
#     bow = bagofword(t)
#     res = model.predict(bow)[0]
#     error_treshold = 0.25
#     result = [[i,r] for i,r in enumerate(res) if r > error_treshold]
    
#     result.sort(key=lambda x: x[1],reverse=True)
#     return_list =[]
#     for r in result:
#         return_list.append({'intents': classes[r[0]], 'probability' : str(r[1])})
#     return return_list

In [9]:
def predict(t):
    bow = bagofword(t)
    result = model.predict(bow)[0]
    print('result : {}'.format(result))
    error_treshold = 0.85
    if result[result_index] >= error_treshold:
        result_index = np.argmax(result)
    else:
        result_index = 'unknown'
    
    return result_index

In [10]:
def get_response(res_index, intent_json):
    if res_index=='unknown':
        result = 'I\'m sorry, I didnt get it'
    else:
        tag = classes[res_index]
        print('tag : {}'.format(tag))
        list_of_intent = intent_json['intents']
        for i in list_of_intent:
            if i['tag'] == tag:
                result=random.choice(i['responses'])
                break
    return result

In [11]:
print('Bot is Running')
while True:
    message = input("Anda : ")
    if message.lower() == "stop" or message.lower() == "quit":
        break
    ints = predict(message)
    response = get_response(ints, intents)
    print('Anda : ',message)
    print('Bot : ',response)

Bot is Running


NotFittedError: Vocabulary not fitted or provided